In [60]:
import pandas as pd
import numpy as np
import xgboost as xgb

%matplotlib inline
%config Completer.use_jedi = False


In [61]:
ds = pd.read_feather('../data/train_for_test.ftr')

In [62]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3646331 entries, 0 to 3646330
Data columns (total 10 columns):
date_block_num           category
shop_id                  category
item_id                  category
item_cnt_month           float64
item_price               float64
year                     category
month                    category
item_cnt_month_lag_1     float64
item_cnt_month_lag_2     float64
item_cnt_month_lag_12    float64
dtypes: category(5), float64(5)
memory usage: 160.8 MB


In [139]:
dsmall=ds[['month','item_cnt_month_lag_1','item_cnt_month_lag_2','item_cnt_month']].dropna()
dsmall.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3423775 entries, 190353 to 3646329
Data columns (total 4 columns):
month                   category
item_cnt_month_lag_1    float64
item_cnt_month_lag_2    float64
item_cnt_month          float64
dtypes: category(1), float64(3)
memory usage: 107.8 MB


In [140]:
dclipped=dsmall.select_dtypes('float').clip(lower=0,upper=20)



month_enc=pd.get_dummies(dsmall.month).values
lags = dclipped[['item_cnt_month_lag_1','item_cnt_month_lag_2']].values

X=np.concatenate((month_enc,lags),axis=1)
y=dclipped.item_cnt_month.values

X

array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [141]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit (X,y )

reg.coef_

array([ -2.90313452e+09,  -2.90313452e+09,  -2.90313452e+09,
        -2.90313452e+09,  -2.90313452e+09,  -2.90313452e+09,
        -2.90313452e+09,  -2.90313452e+09,  -2.90313452e+09,
        -2.90313452e+09,  -2.90313452e+09,  -2.90313452e+09,
         5.06165504e-01,   1.96709275e-01])

In [142]:
reg.score(X,y)

0.41547329081798778

In [143]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y,reg.predict(X)))

1.1232367325678296

In [144]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(X,y)

dtest = xgb.DMatrix(X,y)
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'reg:linear' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

In [145]:
preds

array([ 0.20490706,  0.20490706,  0.20490706, ...,  0.20490706,
        0.20490706,  0.20490706], dtype=float32)

In [146]:
np.sqrt(mean_squared_error(y,np.clip(preds,0,20)))

1.1526401011669725